In [ ]:
pip install crewai==0.80.0

In [1]:
import crewai

In [2]:
crewai.__version__

'0.80.0'

In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv('/home/santhosh/Projects/courses/Pioneer/.env')

True

In [6]:
from crewai import Agent, Task, Crew, LLM

In [7]:
llm = LLM(model="gpt-4o-mini")

# Recipe Recommendation

In [6]:
# User inputs
main_ingredient = "tomato"
dietary_restrictions = "shrimps"

In [6]:
culinary_assistant = Agent(
    llm=llm,
    role="Culinary Assistant",
    backstory="An experienced culinary assistant skilled in finding and tailoring recipes based on ingredients and dietary"
    "needs, and providing clear, step-by-step cooking instructions",
    goal="Find recipes, filter them to meet dietary preferences, and guide user through recipe steps.",
    verbose=True,
)

In [7]:
find_and_filter_recipes = Task(
    description=f"Find recipes that user the ingredient: {main_ingredient} and"
    f" filter them to meet dietary restrictions: {dietary_restrictions}.",
    expected_output=f"One recipe using {main_ingredient} and matching {dietary_restrictions} restrictions.",
    agent=culinary_assistant,
)

guide_recipe_steps = Task(
    description="Provide step-by-step instructions for the selected recipe.",
    expected_output="Step-by-step cooking instructions for the chosen recipe.",
    agent=culinary_assistant,
)


In [8]:
crew = Crew(
    agents=[culinary_assistant],
    tasks=[find_and_filter_recipes, guide_recipe_steps],
    planning=True,
)

In [9]:
crew.kickoff()

# Agent: Culinary Assistant
## Task: Find recipes that user the ingredient: tomato and filter them to meet dietary restrictions: shrimps.Step 1: Understand the Ingredient and Dietary Restriction - Verify the main ingredient: tomato. - Confirm the dietary restriction to be filtered: no shrimp (since the user has a restriction related to shrimps).

Step 2: Conduct a Recipe Search - Utilize culinary databases, cookbooks, or online recipe platforms (e.g., AllRecipes, Food Network, etc.) to initiate a search for recipes that prominently feature tomatoes. - Use search terms such as "tomato recipes" or "dishes with tomato".

Step 3: Filter Results for Dietary Restrictions - From the found recipes, manually or through the searching tool, eliminate any recipes that mention shrimp as an ingredient. - Focus on recipes that highlight tomatoes and do not include shrimp or have shrimp as an optional ingredient.

Step 4: Compile the Suitable Recipes - From the filtered results, choose one recipe that

CrewOutput(raw="Here are the step-by-step cooking instructions for preparing your Tomato Basil Pasta:\n\n### **Ingredients:**\n- 2 cups of fresh tomatoes (diced, can use Roma or Cherry)\n- 3 cups of pasta (spaghetti or your choice; use gluten-free if needed)\n- 2 cloves of garlic (minced)\n- 1/4 cup of fresh basil leaves (chopped)\n- 3 tablespoons of extra virgin olive oil\n- Salt and pepper (to taste)\n- Grated Parmesan cheese (omit for dairy-free; use a substitute if needed)\n\n### **Preparation Steps:**\n\n1. **Cook the Pasta:**\n   - Start by bringing a large pot of salted water to a boil. This will ensure that your pasta is well-seasoned.\n   - Once the water is boiling, add the pasta of your choice. Cook it according to the package instructions until al dente, which usually takes about 8 to 10 minutes depending on the type of pasta.\n   - Before draining, reserve 1 cup of the pasta water, which will help with the sauce. Then drain the pasta and set it aside in a large mixing bowl

In [12]:
crew.usage_metrics

UsageMetrics(total_tokens=4260, prompt_tokens=2296, completion_tokens=1964, successful_requests=3)

# Marketing Research

In [8]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool

In [9]:
product_name = "energy drink"

In [10]:
strategist_backstory = "and marketing strategy"

In [11]:
market_researcher = Agent(
    role="Market Researcher",
    goal="Analyze market trends for the product launch",
    backstory="Experienced in market trends and consumer behavior analysis",
    tools=[SerperDevTool()],
    verbose=True,
)

strategist = Agent(
    role="Product Strategist",
    goal="Create effective positioning strategies for the product",
    backstory=f"Skilled in competitive positioning {strategist_backstory}",
    verbose=True,
)

In [12]:
gather_market_insights_task = Task(
    description=f"Browse the internet to gather insights on current market trends for the launch of the {product_name} product.",
    expected_output=f"List of relevant market trends and consumer preferences, relevant to {product_name}",
    agent=market_researcher,
)

develop_positioning_strategy_task = Task(
    description="Based on the market insights,"
                f"create a positioning strategy for the {product_name} product, including analysis for impact and target audience.",
    expected_output="A positioning strategy with target audience and impact notes",
    agent=strategist,
    verbose=True,
)

In [13]:
crew = Crew(
    agents=[market_researcher, strategist],
    tasks=[gather_market_insights_task, develop_positioning_strategy_task],
    planning=True,
)

In [14]:
output = crew.kickoff()

# Agent: Market Researcher
## Task: Browse the internet to gather insights on current market trends for the launch of the energy drink product.1. Define the Search Query: Determine specific phrases or keywords related to energy drinks and market trends. Use effective search queries like "current market trends energy drinks 2023", "consumer preferences energy drink", and "energy drink market analysis 2023".

2. Access the Tool: Utilize the **Search the Internet** tool (SerperDevTool) and confirm its capability of executing searches.

3. Execute the Search: Input the search query into the tool via the search URL: `https://google.serper.dev/search`.

4. Analyze Results: Review the top results returned by the tool, focusing on articles, reports, and studies that offer insights into market trends and consumer behaviors.

5. Compile Insights: Document key market trends and consumer preferences, such as trends in flavor preferences, packaging styles, marketing approaches, and demographic insi

In [28]:
print(output.raw)

**Positioning Strategy for [Product Name]**

**Executive Summary:**
The lunch products market is evolving rapidly, driven by health-conscious consumer behavior, an emphasis on convenience, and a demand for innovative flavors. Our product, [Product Name], is uniquely positioned to capitalize on these trends by offering a gourmet, health-oriented, and plant-based lunch solution that caters to the modern consumer's needs. This document outlines our target audience, analyzes the market impact, and provides a clear positioning statement that encapsulates how [Product Name] stands out in the competitive landscape.

**1. Target Audience Segmentation:**

We have identified the following key segments that will most benefit from our product:

- **Health-Conscious Women (Ages 25-45)**: Primarily urban dwellers, they prioritize nutrition and seek convenient, healthy meal options that fit their lifestyle. They have a higher disposable income and are willing to pay for premium services.

- **Busy Pr

In [27]:
output.token_usage

UsageMetrics(total_tokens=6099, prompt_tokens=3485, completion_tokens=2614, successful_requests=4)

# Game Flow

In [7]:
import nest_asyncio
nest_asyncio.apply()

In [8]:
import random
from crewai.flow.flow import Flow, listen, router, start
from pydantic import BaseModel

In [9]:
class GameSession(BaseModel):
    player_won: bool = False


class GameFlow(Flow[GameSession]):

    @start()
    def begin_start(self):
        print("Starting the game session")
        player_outcome = random.choice([True, False])
        self.state.player_won = player_outcome

    @router(begin_start)
    def check_outcome(self):
        if self.state.player_won:
            return "win"
        else:
            return "lose"

    @listen("win")
    def celebrate_win(self):
        print("Congratulations!")

    @listen("lose")
    def console_loss(self):
        print("Game Over!")

In [10]:
flow = GameFlow()

In [40]:
flow.kickoff()

Starting the game session


'lose'